In [3]:
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

In [4]:
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

# Membaca data
data = pd.read_csv(r'J:\Drive Saya\sentimen-latih 2\Gojek🏍 App Reviews.csv')

# 1. Data Understanding
# Mengubah semua kolom menjadi tipe objek
data = data.astype('object')

# Menghapus semua data yang kosong
data = data.dropna()

# Menampilkan statistik deskriptif
print(data.describe())

# 2. Data Preprocessing
# Menghapus kolom yang tidak penting
data = data.drop(columns=['at', 'appVersion'])

# Mengubah semua teks menjadi huruf kecil
data['content'] = data['content'].str.lower()

# Normalisasi teks
def normalize_text(text):
    # Variasi penulisan dan typo
    text = re.sub(r'\bgo[- ]?jek\b', 'gojek', text)  # Mengatasi variasi 'gojek', 'go jek', 'go-jek'
    text = re.sub(r'\bgrab[- ]?car\b', 'grabcar', text)  # Mengatasi 'grabcar' dan 'grab car'
    text = re.sub(r'\bgrab[- ]?bike\b', 'grabbike', text)  # Mengatasi 'grabbike' dan 'grab bike'

    # Singkatan dan slang umum
    text = re.sub(r'\bgk\b', 'tidak', text)
    text = re.sub(r'\bga\b', 'tidak', text)
    text = re.sub(r'\bdgn\b', 'dengan', text)
    text = re.sub(r'\bbgt\b', 'banget', text)
    text = re.sub(r'\bdpt\b', 'dapat', text)
    text = re.sub(r'\baja\b', 'saja', text)

    # Konversi angka atau simbol
    text = re.sub(r'5[- ]?star', 'lima bintang', text)  # Mengubah '5-star' atau '5 star' jadi 'lima bintang'
    text = re.sub(r'\b4[- ]?star', 'empat bintang', text)
    text = re.sub(r'\b3[- ]?star', 'tiga bintang', text)
    text = re.sub(r'\b2[- ]?star', 'dua bintang', text)
    text = re.sub(r'\b1[- ]?star', 'satu bintang', text)

    return text

data['content'] = data['content'].apply(normalize_text)

# Tambahkan kata-kata berbobot rendah ke dalam daftar stopwords
stopword_factory = StopWordRemoverFactory()
default_stopwords = stopword_factory.get_stop_words()
stopwords_tambahan = ["saya", "kamu", "kami", "mereka", "itu", "ini", "sudah", "belum", "akan", "bisa", 
                      "dari", "ke", "untuk", "seperti", "tersebut", "dengan", "tanpa", "adalah", "lebih",
                      "kurang", "yang", "dan", "atau", "juga", "hanya", "terlalu", "pada", "oleh", "sehingga",
                      "lagi", "sangat", "lagi", "saat", "dalam", "tentang"]
stopwords_total = set(default_stopwords + stopwords_tambahan)

# Fungsi untuk menghapus stopwords
def remove_stopwords(text):
    tokens = text.split()
    tokens_filtered = [word for word in tokens if word not in stopwords_total]
    return ' '.join(tokens_filtered)

# Menghilangkan stopwords dari konten
data['content'] = data['content'].apply(remove_stopwords)

# Tokenisasi
data['content'] = data['content'].apply(lambda x: x.split())

# Stemming
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()
data['content'] = data['content'].apply(lambda x: [stemmer.stem(word) for word in x])

# Menyimpan data yang sudah dibersihkan
data.to_csv(r'J:\Drive Saya\sentimen-latih 2\Gojek-bersih.csv', index=False)


                                                 userName  \
count                                              225002   
unique                                             210621   
top     Pengguna Google                               ...   
freq                                                 1111   

                                                  content   score  \
count                                              225002  225002   
unique                                             137938       5   
top     Mantap                                        ...       5   
freq                                                 8781  147055   

                          at appVersion  
count                 225002     225002  
unique                224424        145  
top     2023-10-15 13:36:11      4.34.1  
freq                       2      11678  


In [5]:
import pandas as pd
import re

# Membaca data yang telah dibersihkan
data = pd.read_csv(r'J:\Drive Saya\sentimen-latih 2\Gojek-bersih.csv')

# Daftar kata kunci positif dan negatif
kata_positif = ["bagus", "cepat", "puas", "mantap", "nyaman", "baik", "mudah"]
kata_negatif = ["lambat", "buruk", "tidak puas", "jelek", "sulit", "error", "masalah"]

# Fungsi untuk mendeteksi apakah konten mengandung kata positif
def contains_positive(content):
    return any(re.search(r'\b' + re.escape(word) + r'\b', content) for word in kata_positif)

# Fungsi untuk mendeteksi apakah konten mengandung kata negatif
def contains_negative(content):
    return any(re.search(r'\b' + re.escape(word) + r'\b', content) for word in kata_negatif)

# Memisahkan ulasan positif dan negatif berdasarkan skor dan konten
data_positif = data[(data['score'] >= 4) | (data['content'].apply(contains_positive))]
data_negatif = data[(data['score'] < 4) | (data['content'].apply(contains_negative))]

# Menyimpan data ulasan positif ke dalam file CSV
data_positif.to_csv(r'J:\Drive Saya\sentimen-latih 2\Gojek-positif.csv', index=False)

# Menyimpan data ulasan negatif ke dalam file CSV
data_negatif.to_csv(r'J:\Drive Saya\sentimen-latih 2\Gojek-negatif.csv', index=False)


In [5]:
import pandas as pd
from collections import Counter
import ast

# Membaca data
data_negatif = pd.read_csv(r'J:\Drive Saya\sentimen-latih 2\Gojek-negatif.csv')

# Menghapus baris yang kosong atau hanya berisi '[]' di kolom 'content'
data_negatif = data_negatif[data_negatif['content'] != '[]']

# Mengubah kolom 'content' dari string list ke list yang sebenarnya
data_negatif['content'] = data_negatif['content'].apply(ast.literal_eval)

# Menggabungkan semua kata dalam kolom 'content' menjadi satu list besar
all_words = [word for content in data_negatif['content'] for word in content]

# Menghitung frekuensi kata dan menampilkan 10 kata terbanyak
word_counts = Counter(all_words)
top_10_words = word_counts.most_common(10)

# Menampilkan hasil
print("10 kata negatif yang paling sering muncul:")
for word, count in top_10_words:
    print(f"'{word}': {count} kali")


10 kata negatif yang paling sering muncul:
'gojek': 16483 kali
'aplikasi': 14179 kali
'nya': 14107 kali
'driver': 14039 kali
'gak': 10638 kali
'yg': 10613 kali
'mau': 9722 kali
'gopay': 9700 kali
'padahal': 7495 kali
'makin': 7180 kali


In [6]:
import pandas as pd
from collections import Counter
import ast

# Membaca data
data_positif = pd.read_csv(r'J:\Drive Saya\sentimen-latih 2\Gojek-positif.csv')

# Menghapus baris yang kosong atau hanya berisi '[]' di kolom 'content'
data_positif = data_positif[data_positif['content'] != '[]']

# Mengubah kolom 'content' dari string list ke list yang sebenarnya
data_positif['content'] = data_positif['content'].apply(ast.literal_eval)

# Menggabungkan semua kata dalam kolom 'content' menjadi satu list besar
all_words = [word for content in data_positif['content'] for word in content]

# Menghitung frekuensi kata dan menampilkan 10 kata terbanyak
word_counts = Counter(all_words)
top_10_words = word_counts.most_common(10)

# Menampilkan hasil
print("10 kata positif yang paling sering muncul:")
for word, count in top_10_words:
    print(f"'{word}': {count} kali")


10 kata positif yang paling sering muncul:
'bantu': 20246 kali
'gojek': 19992 kali
'bagus': 17796 kali
'mantap': 15392 kali
'aplikasi': 13537 kali
'baik': 13089 kali
'good': 10634 kali
'nya': 10585 kali
'yg': 7739 kali
'mudah': 7686 kali
